<a href="https://colab.research.google.com/github/jonty-s/planets/blob/master/solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np


In [6]:
w0=1
L=1
N=5
mesh=np.linspace(0,L,num=N)
delta=L/(N-1)
X=np.linspace(0,L,num=N)
Y=0.5*np.ones((1,N))
fr=np.ones((1,N))
fi=np.zeros((1,N))
k=1

In [10]:
#finds the discrete derivative of an array arr with step size stepsize
def diff(arr,stepsize):
  n=len(arr)
  d=np.zeros(n)
  if n==1:
    raise ValueError("Can't take a derivative at a single point")
  elif n==2:
    d=[(arr[1]-arr[0])/stepsize,(-arr[0]+arr[1])/stepsize]
    return d
  else:
    d[0]=(-arr[2]/2+2*arr[1]-3*arr[0]/2)/stepsize
    for x in range(1,n-1):
      d[x]= (arr[x+1]-arr[x-1])/(2*stepsize)
    d[-1]=(arr[-3]/2-2*arr[-2]+3*arr[-1]/2)/stepsize
    return d

#defines a useful function we call G, which is part of g
def G(k,w,z):
  x=1/(np.exp(1j*k*z)-np.exp(1j*k*w))
  return x

#defines a useful function we call g
def g(k,z,w):
  x=G(k,-z,w)*G(k,-z,-w)-G(k,z,w)*G(k,z,-w)
  return x

#define the k derivative of g
def g_k(k,z,w):
  x=(1j*z*np.exp(-1j*k*z)+1j*w*np.exp(1j*k*w))*(G(k,-z,w)**2)*G(k,-z,-w) - (-1j*z*np.exp(-1j*k*z)+1j*w*np.exp(-1j*k*w))*G(k,-z,w)*(G(k,-z,-w)**2) + (1j*z*np.exp(1j*k*z)-1j*w*np.exp(1j*k*w))*(G(k,z,w)**2)*G(k,z,-w) + (1j*z*np.exp(1j*k*z)+1j*w*np.exp(-1j*k*w))*G(k,z,w)*(G(k,z,-w)**2)
  return x

#define the z derivative of g
def g_z(k,z,w):
  x=1j*k*np.exp(-1j*k*z)*(G(k,-z,w)**2)*G(k,-z,-w) + 1j*k*np.exp(-1j*k*z)*G(k,-z,w)*(G(k,-z,-w)**2) + 1j*k*np.exp(1j*k*z)*(G(k,z,w)**2)*G(k,z,-w) + 1j*k*np.exp(1j*k*z)*G(k,z,w)*(G(k,z,-w)**2)
  return x

#define the w derivative of g
def g_w(k,z,w):
  x=1j*k*np.exp(1j*k*w)*(G(k,-z,w)**2)*G(k,-z,-w) - 1j*k*np.exp(-1j*k*w)*G(k,-z,w)*(G(k,-z,-w)**2) - 1j*k*np.exp(1j*k*w)*(G(k,z,w)**2)*G(k,z,-w) + 1j*k*np.exp(-1j*k*w)*G(k,z,w)*(G(k,z,-w)**2)
  return x


In [8]:
#first equation we want to be zero is an integral equation
#define the integrand at non-end points
def I(k,Fr,Fi,Xnminus,Xn,Xnplus,Xj,Xjplus,Ynminus,Yn,Ynplus,Yj,Yjplus):
  w=(Xj+Xjplus+1j*(Yj+Yjplus))/2
  firstbit = (Fr+1j*Fi)*(g(k,1j-Xn-1j*Yn,w-1j)-g(k,Xn+1j*Yn,w))*(Xnplus-Xnminus+1j*(Ynplus-Ynminus))/(2*delta)
  secondbit = (Fr-1j*Fi)*(g(k,1j+Xn-1j*Yn,w-1j)-g(k,-Xn+1j*Yn,w))*(Xnplus-Xnminus-1j*(Ynplus-Ynminus))/(2*delta)
  thirdbit = Yn*g(k,1j-Xn-1j*Yn,w-1j)*(Xnplus-Xnminus+1j*(Ynplus-Ynminus))/(2*delta)
  return firstbit+secondbit+thirdbit

#define the integrand at the start point
def Istart(k,Fr,Fi,X0,X1,X2,Xj,Xjplus,Y0,Y1,Y2,Yj,Yjplus):
  w=(Xj+Xjplus+1j*(Yj+Yjplus))/2
  firstbit = (Fr+1j*Fi)*(g(k,1j-X0-1j*Y0,w-1j)-g(k,X0+1j*Y0,w))*(-X2+4*X1-3*X0)+1j*(-Y2+4*Y1-3*Y0)/(2*delta)
  secondbit = (Fr-1j*Fi)*(g(k,1j+X0-1j*Y0,w-1j)-g(k,-X0+1j*Y0,w))*(-X2+4*X1-3*X0)-1j*(-Y2+4*Y1-3*Y0)/(2*delta)
  thirdbit = Y0*g(k,1j-X0-1j*Y0,w-1j)*(-X2+4*X1-3*X0)+1j*(-Y2+4*Y1-3*Y0)/(2*delta)
  return firstbit+secondbit+thirdbit

#define the integrand at the endpoint
def Iend(k,Fr,Fi,Xlast,X2ndlast,X3rdlast,Xj,Xjplus,Ylast,Y2ndlast,Ythirdlast,Yj,Yjplus):
  return -Istart(k,Fr,Fi,Xlast,X2ndlast,X3rdlast,Xj,Xjplus,Ylast,Y2ndlast,Ythirdlast,Yj,Yjplus)

In [12]:
#create jacobian of the system
#the variables are ordered fr[0],...,fr[N-1],fi[0],...,f1[N-1],X[0],...,X[N-1],Y[0],...,Y[N-1],k
#the first 2N-2 equations we want to solve are the integral equations
J=np.zeros((4*N+1,4*N+1))
#first we add the contributions from the frn etc
for j in range(N-1):
  for n in range(N-1):
    J[j][n]=J[j,n]+






[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [20]:
M=[[1,2,3],[4,5,6]]
print(M[0][1])

2
